In [7]:
import csv
import json
import math
import os
import re
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

os.chdir('/Users/cory.quammen/data/PediatricAirways/AtlasDatabase')

In [8]:
def ReadCSVFileIntoDict(filename, header=True):
    d = {}
    with open(filename) as csvFile:
        reader = csv.reader(csvFile, delimiter=',', quotechar='"')
        headerRow = []
        for row in reader:
            if (len(row) > 0 and row[0].startswith('#')):
                continue
            elif (header):
                header = False
                headerRow = row
                #print headerRow
                for col in row:
                    d[col] = []
            else:
                #print row
                if (len(headerRow) > 0):
                    for (name, value) in zip(headerRow, row):
                        d[name].append(value)
                else:
                    for (i,value) in zip(xrange(len(row)), row):
                        colName = 'col%d' % i
                        if (not d.has_key(colName)):
                            d[colName] = []
                        d[colName].append(value)
    return d

In [9]:
# Read all files in the directory
files = os.listdir(os.getcwd())

# Filter the cross section files
files = filter(lambda x: '_ALL_CROSS_SECTIONS.csv' in x, files)
def GetScanID(s):
    m = re.search('[0-9][0-9][0-9][0-9]', s)
    return m.group(0)
scanIDs = map(lambda x: GetScanID(x), files)

In [10]:
# Gather up all curves into a dictionary indexed by scan ID
def float_list(lst):
    return [float(x) for x in lst]

def Distance(x1, x2):
    dx = x1[0] - x2[0]
    dy = x1[1] - x2[1]
    dz = x1[2] - x2[2]
    return math.sqrt(dx*dx + dy*dy + dz*dz)

curveDict = {}
for scanID in scanIDs:
    filename = scanID + '_ALL_CROSS_SECTIONS.csv'
    curveCSV = ReadCSVFileIntoDict(filename)
    centerline = zip(float_list(curveCSV['center of mass:0']),
                     float_list(curveCSV['center of mass:1']),
                     float_list(curveCSV['center of mass:2']))
    cumulativeDistance = [0]
    for (idx, x, y) in zip(xrange(len(centerline)-1), centerline[:-1], centerline[1:]):
        cumulativeDistance.append(Distance(x, y) + cumulativeDistance[idx])
    
    # Normalize the cumulative distance to range [0.0, 1.0]
    parametricDistance = [(t / cumulativeDistance[-1]) for t in cumulativeDistance]
        
    curveDict[scanID] = {'curve' : curveCSV['area'],
                         'centerline' : centerline,
                         't' : parametricDistance}


In [11]:
# Get the patient info
patientInfo = ReadCSVFileIntoDict('PatientInfo.csv')

def CleanNumberString(s):
    return ''.join([c for c in s if c in '0123456789-.'])

for (scanID, age) in zip(patientInfo['Number'], patientInfo['Age in months']):
    if (not curveDict.has_key(scanID)):
        continue
    curveDict[scanID]['age'] = float(CleanNumberString(age))

# Clean out curves for which we don't have patient data
for key in curveDict.keys():
    if (not curveDict[key].has_key('age')):
        print "Could not find age for scan", key
        print "Removing from database"
        curveDict.pop(key, None)

Could not find age for scan 1024
Removing from database


In [12]:
# Find index of point on curve closest to landmark
landmarksOfInterest = ['TVC', 'Subglottis', 'EpiglottisTip', 'TracheaCarina']
for scanID in curveDict.keys()[2:]:
    # Find the landmark file for the scan
    filename = scanID + '_LANDMARKS.fcsv'
    with open(filename) as landmarksFile:        
        landmarksCSV = ReadCSVFileIntoDict(filename, header=False)
    xcol = [-float(x) for x in landmarksCSV['col1']]
    ycol = [-float(x) for x in landmarksCSV['col2']]
    zcol = [float(x) for x in landmarksCSV['col3']]
    labelcol = landmarksCSV['col11']
    curveDict[scanID]['landmarks'] = {}
    for (x, y, z, label) in zip(xcol, ycol, zcol, labelcol):
        if (label in landmarksOfInterest):
            # Find curve point nearest to landmark
            x = float(x)
            y = float(y)
            z = float(z)
            landmarkPosition = (x, y, z)
            centerline = curveDict[scanID]['centerline']
            distances = map(lambda x: Distance(landmarkPosition, x), centerline)
            minDist = min(distances)
            minDistIndex = distances.index(minDist)

            curveDict[scanID]['landmarks'][label] = {'closest_index' : minDistIndex,
                                                     'position' : landmarkPosition}


In [34]:
# Get average parametric coordinate of the various landmarks
averageT = {}
for landmark in landmarksOfInterest:
    averageT[landmark] = {'cumulative' : 0.0,
                          'count' : 0}
    for scanID in curveDict.keys():
        curveInfo = curveDict[scanID]
        try:
            landmarkDict = curveInfo['landmarks'][landmark]
        except:
            curveDict.pop(scanID, None)
            continue
        tIndex = landmarkDict['closest_index']
        t = curveInfo['t'][tIndex]
        averageT[landmark]['cumulative'] += t
        averageT[landmark]['count'] += 1

    averageT[landmark] = averageT[landmark]['cumulative'] / averageT[landmark]['count']

print averageT


{'Subglottis': 0.8235214658659263, 'TracheaCarina': 0.9673666744737812, 'EpiglottisTip': 0.6924648456880057, 'TVC': 0.7909759943093555}
